<a href="https://colab.research.google.com/github/Boga28/Google-Colab/blob/main/AutoTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#@markdown #Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown Install Library

%%bash
set -e

cd "$(dirname "${BASH_SOURCE[0]}")"

if [[ "$(pwd)" =~ " " ]]; then
    echo "This script relies on Miniconda which cannot be silently installed under a path with spaces."
    exit 1
fi

# Deactivate existing conda environments to avoid conflicts.
{ conda deactivate && conda deactivate && conda deactivate; } 2> /dev/null

OS_ARCH=$(uname -m)
case "${OS_ARCH}" in
    x86_64*)    OS_ARCH="x86_64" ;;
    arm64* | aarch64*) OS_ARCH="aarch64" ;;
    *)          echo "Unknown system architecture: $OS_ARCH! This script runs only on x86_64 or arm64" && exit 1 ;;
esac

# Enable universe repository and install Linux requirements.
sudo add-apt-repository universe
sudo apt update
sudo apt install -y libaio-dev espeak-ng ffmpeg gcc g++

# Configuration paths.
INSTALL_DIR="$(pwd)/xttsv2"
CONDA_ROOT_PREFIX="${INSTALL_DIR}/conda"
INSTALL_ENV_DIR="${INSTALL_DIR}/env"
MINICONDA_DOWNLOAD_URL="https://repo.anaconda.com/miniconda/Miniconda3-py311_24.4.0-0-Linux-${OS_ARCH}.sh"

if [ ! -x "${CONDA_ROOT_PREFIX}/bin/conda" ]; then
    echo "Downloading Miniconda from ${MINICONDA_DOWNLOAD_URL} to ${INSTALL_DIR}/miniconda_installer.sh"
    mkdir -p "${INSTALL_DIR}"
    curl -L "${MINICONDA_DOWNLOAD_URL}" -o "${INSTALL_DIR}/miniconda_installer.sh"
    chmod +x "${INSTALL_DIR}/miniconda_installer.sh"
    bash "${INSTALL_DIR}/miniconda_installer.sh" -b -p "${CONDA_ROOT_PREFIX}"
    echo "Miniconda installed."
fi

if [ ! -d "${INSTALL_ENV_DIR}" ]; then
    "${CONDA_ROOT_PREFIX}/bin/conda" create -y --prefix "${INSTALL_ENV_DIR}" -c conda-forge python=3.11.9
    echo "Conda environment created at ${INSTALL_ENV_DIR}."
fi

# Activate the conda environment and install Python requirements.
source "${CONDA_ROOT_PREFIX}/etc/profile.d/conda.sh"
conda activate "${INSTALL_ENV_DIR}"

pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121
conda install -y nvidia/label/cuda-12.1.0::cuda-toolkit=12.1
conda install -y pytorch::faiss-cpu
conda install -y -c conda-forge "ffmpeg=*=*gpl*"
conda install -y -c conda-forge "ffmpeg=*=h*_*" --no-deps
echo "Fixing Nvidia's broken symlinks in the env/lib folder..."
env_path="${INSTALL_ENV_DIR}/lib"
echo "Installing additional requirements..."
pip install coqui-tts[all]
echo "Installing DeepSpeed..."
wget https://github.com/erew123/alltalk_tts/releases/download/DeepSpeed-14.2-Linux/deepspeed-0.14.2+cu121torch2.3-cp311-cp311-manylinux_2_24_x86_64.whl
pip install deepspeed-0.14.2+cu121torch2.3-cp311-cp311-manylinux_2_24_x86_64.whl
conda clean --all --force-pkgs-dirs -y

cd "$(dirname "${BASH_SOURCE[0]}")"
if [[ "$(pwd)" =~ " " ]]; then
    echo "This script relies on Miniconda which cannot be silently installed under a path with spaces."
    exit 1
fi

{ conda deactivate && conda deactivate && conda deactivate; } 2> /dev/null

CONDA_ROOT_PREFIX="$(pwd)/xttsv2/conda"
INSTALL_ENV_DIR="$(pwd)/xttsv2/env"
export PYTHONNOUSERSITE=1
unset PYTHONPATH
unset PYTHONHOME
export CUDA_PATH="$INSTALL_ENV_DIR"
export CUDA_HOME="$CUDA_PATH"

bash --init-file <(echo "source \"$CONDA_ROOT_PREFIX/etc/profile.d/conda.sh\" && conda activate \"$INSTALL_ENV_DIR\"")


In [ ]:
#@markdown Download MODEL

# Create a directory for the XTTS-v2 model files and move into it.
!mkdir -p xtts2_model
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth -O /content/xtts2_model/model.pth
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/config.json -O /content/xtts2_model/config.json
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json -O /content/xtts2_model/vocab.json
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/hash.md5 -O /content/xtts2_model/hash.md5
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/speakers_xtts.pth -O /content/xtts2_model/speakers_xtts.pth

# CLONE
!wget https://raw.githubusercontent.com/Boga28/Google-Colab/refs/heads/main/Christopher_lee_clean.wav -O /content/Christopher_lee_clean.wav
!wget https://github.com/Boga28/Google-Colab/raw/refs/heads/main/eTTs.py -O /content/eTTs.py
!wget https://github.com/Boga28/Google-Colab/raw/refs/heads/main/input.txt -O /content/input.txt



In [113]:
#@markdown Run Script

%%bash
# (Optional) Deactivate any active environments.
{ conda deactivate && conda deactivate && conda deactivate; } 2> /dev/null

# Define paths.
CONDA_ROOT_PREFIX="/content/xttsv2/conda/"
INSTALL_ENV_DIR="/content/xttsv2/env"

# Export necessary variables.
export PYTHONNOUSERSITE=1
unset PYTHONPATH
unset PYTHONHOME
export CUDA_PATH="$INSTALL_ENV_DIR"
export CUDA_HOME="$CUDA_PATH"
export PATH=$CUDA_HOME/bin:$PATH

# Set a valid matplotlib backend.
export MPLBACKEND=agg

# Source the conda setup script and activate the environment.
source "$CONDA_ROOT_PREFIX/etc/profile.d/conda.sh"
conda activate "$INSTALL_ENV_DIR"
#pip install coqui-tts[all]
#pip install /content/deepspeed-0.14.2+cu121torch2.3-cp311-cp311-manylinux_2_24_x86_64.whl

# Run your Python script.
python /content/eTTs.py

INFO: activate-binutils_linux-64.sh made the following environmental changes:
+ADDR2LINE=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-addr2line
+AR=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-ar
+AS=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-as
+CXXFILT=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-c++filt
+ELFEDIT=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-elfedit
+GPROF=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-gprof
+HOST=x86_64-conda_cos6-linux-gnu
+LD=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-ld
+LD_GOLD=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-ld.gold
+NM=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-nm
+OBJCOPY=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-objcopy
+OBJDUMP=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-objdump
+RANLIB=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-ranlib
+READELF=/content/xttsv2/env/bin/x86_64-conda_cos6-linux-gnu-readelf
+SIZE=/content/xttsv2/env/bin/x86